In [1]:
from demo  import jdt_db
import pandas as pd
import datetime
import time
import  numpy  as np 

# 0.0时间计算


In [2]:
### 30天时间计算
today_time = datetime.datetime.now()
last_month_today_time = today_time - datetime.timedelta(days = 30)
yesterday_time = today_time - datetime.timedelta(days = 1)
lastyerday_time = today_time - datetime.timedelta(days = 2)

In [4]:
lastyerday_time

datetime.datetime(2022, 5, 31, 16, 44, 37, 675764)

In [3]:
today = today_time.strftime("%Y-%m-%d ") + "00:00:00"
last_month_today = last_month_today_time.strftime("%Y-%m-%d ") + "00:00:00"
yesterday = yesterday_time.strftime("%Y-%m-%d ") + "00:00:00"
lastyerday = lastyerday_time.strftime("%Y-%m-%d ") + "00:00:00"
out_time = (datetime.datetime.now() - datetime.timedelta(days = 365)).strftime("%Y-%m-%d ") + "00:00:00"

In [5]:
lastyerday

'2022-05-31 00:00:00'

In [7]:
sql_today = "'" + today + "'"
sql_last_month_today ="'" + last_month_today + "'"
sql_yesterday ="'" + yesterday + "'"

In [8]:
sql_yesterday

"'2022-06-01 00:00:00'"

# 1.1  昨日前日用户新增


In [3]:
### 30天数据导出
sql1 = "SELECT	a.id,	FROM_UNIXTIME( a.jointime, '%Y-%m-%d %H:%i:%s' ) AS '加入时间',	a.p_name AS '父级名称',	a.gender AS '性别',	a.age AS '年龄',	a.birthday AS '生日',	a.province AS '地区编码',	a.user_province AS '注册省份',	a.user_city AS '注册城市',	b.conference_id,	b.c AS '最近参加会务时间',	b.d AS '参加会务次数' ,	g.group_id as '用户级别',	g.f as '用户最后登陆时间',g.e as '用户登录次数'  FROM	jdt_user AS a	LEFT JOIN (	SELECT		conference_id,		buyer_id,		MAX( created_at ) AS c,		count( buyer_id ) AS d 	FROM		jdt_conference_order 	WHERE		created_at >=" + sql_last_month_today + "	AND created_at < " + sql_today + " 	GROUP BY buyer_id) AS b ON b.buyer_id = a.id LEFT JOIN (SELECT user_id,group_id,MAX(created_at) as f, count(user_id) as e FROM jdt_user_logon_in WHERE created_at >= " + sql_last_month_today +  "	AND created_at < "+ sql_today +  	"GROUP BY	user_id) as g on g.user_id = a.id  WHERE FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) >= "+ sql_last_month_today + "	AND FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) <" +  sql_today 
data1 = jdt_db(sql1)
###  30天数据清洗与导出
data1['conference_id'] = data1['conference_id'].fillna(0)
data1.to_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\1.新增用户数据.csv')

23.61221480369568


In [4]:
### 前两天的数据导出
yesterday_data = data1[(data1['加入时间']  >= yesterday) &  (data1['加入时间']  < today)]
lastyerday_data = data1[(data1['加入时间']  >= lastyerday) &  (data1['加入时间']  < yesterday)]

In [5]:
### 前两天数据计算
yesterday_data_up_count = yesterday_data['id'].count()
lastyerday_data_up_count = lastyerday_data['id'].count()
yesterday_lastyerday_up_count = yesterday_data_up_count - lastyerday_data_up_count
yesterday_data_conference_count = yesterday_data[yesterday_data['conference_id'] > 0 ]['id'].count()

# 1.2  昨日前日活跃用户人数，活跃用户次数

In [6]:
# 昨日用户活跃数据


sql2 = "SELECT user_id, group_id, province,city,created_at,updated_at  FROM  jdt_user_logon_in   WHERE   updated_at  >" +sql_yesterday + " and updated_at  <  "+ sql_today 
data2 = jdt_db(sql2)
data2['des'] = np.where(data2['updated_at']>data2['created_at'],'重复',np.where(data2['updated_at']==data2['created_at'],'新增','错误'))
data2.to_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\2.用户活跃数据.csv')

3.7626609802246094


In [7]:
### 活跃次数、人数
active_times = data2['user_id'].count() 
active_pople = data2.drop_duplicates(subset = 'user_id',keep = 'last')['user_id'].count()
print('活跃用户次数   ' + str(active_times))
print('活跃用户人数   ' + str(active_pople))

活跃用户次数   6150
活跃用户人数   4510


# 1.3  昨日升级数量，新用户升级数量

In [8]:
# 昨日用户活跃数据

sql3 = "SELECT  a.user_id, a.created_at, a.user_upgrade_group, a.money , FROM_UNIXTIME( b.jointime, '%Y-%m-%d %H:%i:%s' )  as '加入时间'  from  jdt_user_upgrade_log  as a LEFT JOIN jdt_user as b on a.user_id = b.id  WHERE a.created_at  > "  +sql_yesterday + "AND   a.created_at  < " +sql_today  + 'order by user_id'         
data3 = jdt_db(sql3)
data3['created_at'] =pd.to_datetime(data3['created_at']).dt.date
data3['加入时间'] = pd.to_datetime(data3['加入时间']).dt.date
data3['des'] = np.where(data3['加入时间'] == data3['created_at'],'同日','非同')
data3.to_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\3.用户升级数据.csv')

0.35562586784362793


In [9]:
# 升级数据
up_pople_sum = data3['user_id'].count()
up_pople_equally = data3[data3['des'] == '同日']['user_id'].count()
up_pople_different = data3[data3['des'] == '非同']['user_id'].count()
conference_money = data3['money'].sum()
print('升级总人数   ' + str(up_pople_sum))
print('新用户注册就升级  ' + str(up_pople_equally))
print('用户注册后升级   ' + str(up_pople_different))
print('升级总金额    ' + str(conference_money))

升级总人数   410
新用户注册就升级  85
用户注册后升级   325
升级总金额    1898441.0


# 1.4 新建立会务数、参加会务人数


In [10]:
# 昨日开始会务量
sql4 = "SELECT  id,conference_name,is_offline from  jdt_conference  WHERE   created_at  > "  +sql_yesterday + "AND    created_at  < " +sql_today 
data4 = jdt_db(sql4)
conference_num = data4['id'].count()
# 昨日会务用户数量
sql5 = "SELECT  buyer_id,buyer_name   from  jdt_conference_order   WHERE  created_at > "  +sql_yesterday + "AND    created_at  <" +sql_today 
con_people = jdt_db(sql5)['buyer_id'].count()

0.1327500343322754
0.22217130661010742


# 1.5每日具体升学个级别人数

In [11]:
sql5 = "SELECT   CASE  user_upgrade_group WHEN '1' THEN 'circle1' WHEN '2' THEN 'circle2' WHEN '3' THEN 'circle3' WHEN '4' THEN 'circle4' WHEN '5' THEN 'circle5' ELSE '其他' END AS 'circle', count(user_id) as 'count'  FROM  jdt_user_upgrade_log  WHERE   created_at  > "  +sql_yesterday + "AND    created_at  < " +sql_today + "GROUP BY  user_upgrade_group"
data5 = jdt_db(sql5)

data_circle = pd.read_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\4.circle升级数据.csv',encoding='gbk')
del data_circle['count']
del data_circle['size']
del data_circle['Unnamed: 0']
data_circle=pd.merge(data_circle,data5,on='circle')

data_circle['size']=data_circle['count'] /data_circle.iloc[0, 3]
data_circle.to_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\4.circle升级数据.csv',encoding='gbk')
data_circle

0.44943785667419434


,circle,R,link,count,size
0,circle2,2.0,link,162,1.000000
1,circle3,1.5,link,232,1.432099
2,circle4,1.0,link,16,0.098765


In [12]:
###  生成数据字典


# zuo_time   时间
# qian_data_up_count    前日新增
# zuo_data_up_count     昨天新增
# zuo_qian_up_count     比较前日增长
# zuo_data_conference_count   会务新用户
# ------------------------------------------------------------------------
Every_day_dict = {
                  '时间':yesterday,  
                  '前日新增用户人数':lastyerday_data_up_count,
                  '昨天新增用户人数':yesterday_data_up_count,
                  '较前日新增人增长':yesterday_lastyerday_up_count,
                  '参加会务新用户数':yesterday_data_conference_count,
    '用户活跃次数':active_times,
    '用户活跃人数':active_pople,
        '新用户注册就升级人数':up_pople_equally,
        '升级会务总营收金额':conference_money,
    '昨日新开会务数':conference_num,
    '昨日参加会务总数':con_people,
    '昨日升级总用户数':up_pople_sum
                    }

Every_day_dict


{'时间': '2022-05-15 00:00:00',
 '前日新增用户人数': 954,
 '昨天新增用户人数': 960,
 '较前日新增人增长': 6,
 '参加会务新用户数': 250,
 '用户活跃次数': 6150,
 '用户活跃人数': 4510,
 '新用户注册就升级人数': 85,
 '升级会务总营收金额': 1898441.0,
 '昨日新开会务数': 45,
 '昨日参加会务总数': 6549,
 '昨日升级总用户数': 410}

In [13]:
### 每日数据保存

Every_day_data = pd.DataFrame(Every_day_dict,index=[0])

sum_every_data = pd.read_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\sum_every_data.csv',encoding='gbk')
sum_every_data = sum_every_data.drop(columns=['Unnamed: 0'],axis=1)

sum_every_data = pd.concat([sum_every_data,Every_day_data],axis=0)
sum_every_data = sum_every_data[sum_every_data['时间']>= out_time]
sum_every_data.drop_duplicates(subset=['时间'], keep='last', inplace=True)
sum_every_data.to_csv(r'D:\DTy_work\analysis\2022.05.07数据每日看板详情\data\sum_every_data.csv',encoding='gbk')


In [14]:
sum_every_data

,时间,前日新增用户人数,昨天新增用户人数,较前日新增人增长,参加会务新用户数,用户活跃次数,用户活跃人数,新用户注册就升级人数,升级会务总营收金额,昨日新开会务数,昨日参加会务总数,昨日升级总用户数
0,2022-05-06 00:00:00,1126,968,-158,340,4760,3539,79,784936.0,82,8509,370
1,2022-05-08 00:00:00,1228,1998,770,645,12375,9104,130,2241869.0,96,15868,586
2,2022-05-09 00:00:00,1998,1013,-985,301,5203,3941,65,866547.0,81,18168,209
0,2022-05-15 00:00:00,954,960,6,250,6150,4510,85,1898441.0,45,6549,410
